* [Compiling stages](#Compiling-stages)
    - [Explicitly Compilation with distutils](#Explicitly-Compilation-with-distutils)
    - [Implicitly Compilation with Jupyter](#Implicitly-Compilation-with-Jupyter)
* [Cython Typing](#Cython-Typing)
    - [Poiters](#Poiters)
    - [Statically Declaring Variables with a Python Type](#Statically-Declaring-Variables-with-a-Python-Type)
* [C division vs Cython Division](#C-division-vs-Cython-Division)
* [Functions](#Functions)
    - [Combining def and cdef Functions with cpdef](#Combining-def-and-cdef-Functions-with-cpdef)
    - [Dealing with exceptions](#Dealing-with-exceptions)

Anotations taked from [Cython: A Guide for Python Programmers](https://www.amazon.com.br/Cython-Kurt-Smith/dp/1491901551/ref=asc_df_1491901551/?tag=googleshopp00-20&linkCode=df0&hvadid=379735814613&hvpos=&hvnetw=g&hvrand=18010478823698863835&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1001706&hvtargid=pla-456029009508&psc=1)

# Compiling stages


The pipeline comprises two stages. The first stage is handled by the cython compiler, which transforms Cython source into optimized and platform-independent C or C++.
The second stage compiles the generated C or C++ source into a shared library with a standard C or C++ compiler. The resulting shared library is platform dependent. It is a shared object file with a *.so* extension on Linux or Mac OS X, and is a dynamic library
with a *.pyd* extension on Windows.

This pipeline can be run either automatically, without user involvement (making Cython feel much like Python), or explicitly by the end user when more control is required.


## Explicitly Compilation with distutils

```
~$ python setup.py build_ext --inplace
```

> The cython compiler is a source-to-source compiler, and the generated code is highly optimized. It is not uncommon for Cython generated C code to be faster than typical hand-written C. When the author teaches Cython, students often write C equivalents to Cython’s code; the Cython version is nearly always faster, and for equivalent algorithms is never slower. Cython’s generated C code is also highly portable, supporting all common C compilers and many Python versions simultaneously. 

*book "Cython: a guide for Python Programmers" pg. 13*

## Implicitly Compilation with Jupyter

Jupyter has several magic commands (click [here](https://notebook.community/acrispin/cython/docs/examples/Cython%20Magics) to check them) to allow dynamic compilation of Cython code.

In [1]:
%load_ext cython

In [7]:
%%cython
def fib(int n):
    cdef int i
    cdef double a=0.0 
    cdef double b=1.0
    for i in range(n):
        a, b = a + b, a
    return a

In [8]:
fib(90)

2.880067194370816e+18

# Cython Typing

![](imgs/c_types.png)

Type declaration could be in the following way:

In [15]:
%%cython
cdef int i
cdef int N=2000
cdef float dx, s=0.0

or:

In [17]:
%%cython
cdef:
    int i, N=2000
    float dx, s=0.0

To enable type inference for a function, we can use the decorator form of infer_types:

In [20]:
%%cython
cimport cython

@cython.infer_types(True)
def more_inference():
    i = 1
    d = 2.0
    c = 3+4j
    r = i * d + c
    return r

## Poiters

Pointers can be declared in this way:

In [23]:
%%cython
cdef int *a, *b

Or this way:

In [25]:
%%cython
cdef int* a, b

Dereferencing pointers in Cython is different than in C. Because the Python language already uses the *args and **kwargs syntax to allow arbitrary positional and keyword arguments and to support function argument unpacking, Cython does not support the *a syntax to dereference a C pointer. Instead, we index into the pointer at location 0 to dereference a pointer in Cython. This syntax also works to dereference a pointer in C.

In [14]:
%%cython
cdef double golden_ratio
cdef double *p_double

p_double = &golden_ratio

p_double[0] = 1.618
print(golden_ratio)

1.618


## Statically Declaring Variables with a Python Type

Not all Python types can be statically declared: they must be implemented in C and
Cython must have access to the declaration. 

The built-in Python types already satisfy
these requirements, and declaring them is straightforward

In [26]:
%%cython
cdef list particles, modified_particles
cdef dict names_from_particles
cdef str pname
cdef set unique_particles

# C division vs Cython Division

C and Python have markedly different behavior when computing. Cython uses Python semantics by default.

To obtain C semantics, we can use the cdivision compiler directive, either at the global module level, or in a directive comment ```# cython: cdivision=True```:

In [32]:
%%cython
cimport cython

@cython.cdivision(True)
def divides(int a, int b):
    return a / b

In [29]:
divides(-1, 2)

0

In [31]:
divides(10, 2)

5

Or within a function with a context manager:

In [33]:
%%cython
cimport cython

def remainder(int a, int b):
    with cython.cdivision(True):
        return a % b

In [34]:
remainder(5, 3)

2

# Functions

Cython supports both Python and C functions and allows them to call each other in a
natural and straightforward way, all in the same source file.

Cython supports regular Python functions defined with the def keyword, and they work
as we would expect.

In [34]:
%%cython

# Python visible function signature:
def fact_py(n):
    """Computes n!"""
    if n <= 1:
        return 1
    return n * fact_py(n - 1)

In [30]:
%timeit fact_py(20)

945 ns ± 65 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Just compile a python function in Cython make a difference. Lets define a interpreted python function to compare:

In [11]:
def interpred_fact(n):
    """Computes n!"""
    if n <= 1:
        return 1
    return n * interpred_fact(n - 1)

In [31]:
%timeit interpred_fact(20)

2.3 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


The interpreted onde is double  slower then the py compilated onde.

When used to define a function, the cdef keyword creates a function with C-calling
semantics. A cdef function’s arguments and return type are typically statically typed,
and they can work with C pointer objects, structs, and other C types that cannot be
automatically coerced to Python types. It is helpful to think of a cdef function as a C
function that is defined with Cython’s Python-like syntax.

In [35]:
%%cython
# Not reachable in pure python:
cdef long fact_cy(long n):
    """Computes n!"""
    if n <= 1:
        return 1
    return n * fact_cy(n - 1)

# wrapper around the cython function:
def wrapp_fact_cy(long n):
    return fact_cy(n)

In [33]:
%timeit wrapp_fact_cy(20)

58.7 ns ± 2.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Unfortunately, the wrapp_fact_cy function comes with some limitations. One limitation
is that wrapp_fact_cy and its underlying fact_cy are restricted to C integral types only,
and do not have the benefit of Python’s unlimited-precision integers. In practice, this
means that wrapp_fact_cy gives erroneous results for arguments larger than some small
value, depending on how large an unsigned long is on our system.

## Combining def and cdef Functions with cpdef

A single cpdef function gives us these two functions automatically: we get a C-only version
of the function and a Python wrapper for it with the same name.

In [36]:
%%cython
cpdef long cp_fact(long n):
    """Computes n!"""
    if n <= 1:
        return 1
    return n * cp_fact(n - 1)

In [37]:
%timeit cp_fact(20)

63.9 ns ± 3.56 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


A cpdef function has one limitation, due to the fact that it does double duty as both a
Python and a C function: its arguments and return types have to be compatible with
both Python and C types. Any Python object can be represented at the C level (e.g., by
using a dynamically typed argument, or by statically typing a built-in type), but not all
C types can be represented in Python. So, we cannot use void, C pointers, or C arrays
indiscriminately as the argument types or return type of cpdef functions.

## Dealing with exceptions

A def function always returns some sort of PyObject pointer at the C level. This invar‐
iant allows Cython to correctly propagate exceptions from def functions without is‐
sue. Cython’s other two function types—cdef and cpdef—may return a non-Python
type, which makes some other exception-indicating mechanism necessary.

In [5]:
%%cython

cpdef double divide_ints(int i, int j) except? -1:
    return i / j

In [6]:
divide_ints(1, 2)

0.5

In [7]:
divide_ints(1, 0)

ZeroDivisionError: float division

The except? -1 clause allows the return value -1 to act as a possible sentinel that an
exception has occurred. The value -1 here is arbitrary: we could have used a different integer
literal that is within the range of values for the return type.

If there is a return value that always indicates an error has occurred without ambiguity, then the question
mark can be omitted. Alternatively, to have Cython check if an exception has been raised
regardless of return value, we can use the except * clause instead. This will incur some
overhead.